In [17]:
import pandas as pd
import requests
import time

# === CONFIG ===
API_KEY = "PJcSVU8gR4wG9S9HVANWyssG62n2M6Sy"
TICKERS_URL = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
LIMIT = 10  # number of past reports (1 = latest)
SLEEP_TIME = 1  # avoid rate limit

# === LOAD TICKERS ===
print("Loading S&P 500 tickers...")
tickers_df = pd.read_csv(TICKERS_URL)
tickers = tickers_df['Symbol'].tolist()

# === FUNCTION TO FETCH DATA ===
def get_fmp_data(endpoint, symbol):
    url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{symbol}?limit={LIMIT}&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return []

# === MAIN LOOP ===
all_data = []

print("Fetching data for each ticker...")
for i, symbol in enumerate(tickers[0:70]):
    print(f"[{i+1}/{len(tickers)}] {symbol}")

    income_list = get_fmp_data("income-statement", symbol)
    balance_list = get_fmp_data("balance-sheet-statement", symbol)
    cashflow_list = get_fmp_data("cash-flow-statement", symbol)

    for idx in range(LIMIT):
        income = income_list[idx] if idx < len(income_list) else {}
        balance = balance_list[idx] if idx < len(balance_list) else {}
        cashflow = cashflow_list[idx] if idx < len(cashflow_list) else {}

        if income or balance or cashflow:
            merged = {
                "symbol": symbol,
                "reportIndex": idx + 1,
                "reportDate": income.get("date", balance.get("date", cashflow.get("date", None))),
                **income,
                **{f"balance_{k}": v for k, v in balance.items()},
                **{f"cashflow_{k}": v for k, v in cashflow.items()}
            }
            all_data.append(merged)

    time.sleep(SLEEP_TIME)

# === SAVE RESULTS ===
df_0_70 = pd.DataFrame(all_data)
df_0_70.to_parquet("0_70_sp500_fundamentals.parquet", index=False)
print("✅ Data saved to '0_70_sp500_fundamentals.parquet'")


Loading S&P 500 tickers...
Fetching data for each ticker...
[1/503] MMM
[2/503] AOS
[3/503] ABT
[4/503] ABBV
[5/503] ACN
[6/503] ADBE
[7/503] AMD
[8/503] AES
[9/503] AFL
[10/503] A
[11/503] APD
[12/503] ABNB
[13/503] AKAM
[14/503] ALB
[15/503] ARE
[16/503] ALGN
[17/503] ALLE
[18/503] LNT
[19/503] ALL
[20/503] GOOGL
[21/503] GOOG
[22/503] MO
[23/503] AMZN
[24/503] AMCR
[25/503] AEE
[26/503] AEP
[27/503] AXP
[28/503] AIG
[29/503] AMT
[30/503] AWK
[31/503] AMP
[32/503] AME
[33/503] AMGN
[34/503] APH
[35/503] ADI
[36/503] ANSS
[37/503] AON
[38/503] APA
[39/503] APO
[40/503] AAPL
[41/503] AMAT
[42/503] APTV
[43/503] ACGL
[44/503] ADM
[45/503] ANET
[46/503] AJG
[47/503] AIZ
[48/503] T
[49/503] ATO
[50/503] ADSK
[51/503] ADP
[52/503] AZO
[53/503] AVB
[54/503] AVY
[55/503] AXON
[56/503] BKR
[57/503] BALL
[58/503] BAC
[59/503] BAX
[60/503] BDX
[61/503] BRK.B
[62/503] BBY
[63/503] TECH
[64/503] BIIB
[65/503] BLK
[66/503] BX
[67/503] BK
[68/503] BA
[69/503] BKNG
[70/503] BSX
✅ Data saved to '0_70

In [18]:
df_0_70.shape

(350, 134)

In [21]:
df_0_70.head(15)

,symbol,reportIndex,reportDate,date,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,...,cashflow_netCashUsedProvidedByFinancingActivities,cashflow_effectOfForexChangesOnCash,cashflow_netChangeInCash,cashflow_cashAtEndOfPeriod,cashflow_cashAtBeginningOfPeriod,cashflow_operatingCashFlow,cashflow_capitalExpenditure,cashflow_freeCashFlow,cashflow_link,cashflow_finalLink
0,MMM,1,2024-12-31,2024-12-31,USD,0000066740,2025-02-05,2025-02-05 09:57:15,2024,FY,...,1098000000,-44000000.0,-333000000,5600000000,5933000000,1819000000,-1181000000,638000000,https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...
1,MMM,2,2023-12-31,2023-12-31,USD,0000066740,2024-02-07,2024-02-07 09:25:16,2023,FY,...,-3147000000,-48000000.0,2278000000,5933000000,3655000000,6680000000,-1615000000,5065000000,https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...
2,MMM,3,2022-12-31,2022-12-31,USD,0000066740,2023-02-08,2023-02-08 12:59:49,2022,FY,...,-5350000000,-104000000.0,-909000000,3655000000,4564000000,5591000000,-1749000000,3842000000,https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...
3,MMM,4,2021-12-31,2021-12-31,USD,0000066740,2022-02-09,2022-02-09 15:13:29,2021,FY,...,-6145000000,-62000000.0,-70000000,4564000000,4634000000,7454000000,-1603000000,5851000000,https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...
4,MMM,5,2020-12-31,2020-12-31,USD,0000066740,2021-02-04,2021-02-04 13:53:11,2020,FY,...,-5300000000,48000000.0,2281000000,4634000000,2353000000,8113000000,-1501000000,6612000000,https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...
5,AOS,1,2024-12-31,2024-12-31,USD,0000091142,2025-02-11,2025-02-11 17:28:15,2024,FY,...,-408400000,-6600000.0,-100300000,239600000,339900000,581800000,-108000000,473800000,https://www.sec.gov/Archives/edgar/data/91142/...,https://www.sec.gov/Archives/edgar/data/91142/...
6,AOS,2,2023-12-31,2023-12-31,USD,0000091142,2024-02-13,2024-02-13 16:46:51,2023,FY,...,-684700000,-12800000.0,-51300000,339900000,391200000,670300000,-72600000,597700000,https://www.sec.gov/Archives/edgar/data/91142/...,https://www.sec.gov/Archives/edgar/data/91142/...
7,AOS,3,2022-12-31,2022-12-31,USD,0000091142,2023-02-14,2023-02-14 17:11:05,2022,FY,...,-430800000,-20800000.0,-52100000,391200000,443300000,391400000,-70300000,321100000,https://www.sec.gov/Archives/edgar/data/91142/...,https://www.sec.gov/Archives/edgar/data/91142/...
8,AOS,4,2021-12-31,2021-12-31,USD,0000091142,2022-02-11,2022-02-11 14:19:32,2021,FY,...,-421000000,0.0,-129800000,443300000,573100000,641100000,-75100000,566000000,https://www.sec.gov/Archives/edgar/data/91142/...,https://www.sec.gov/Archives/edgar/data/91142/...
9,AOS,5,2020-12-31,2020-12-31,USD,0000091142,2021-02-12,2021-02-12 12:27:08,2020,FY,...,-374800000,0.0,199100000,573100000,374000000,562100000,-56800000,505300000,https://www.sec.gov/Archives/edgar/data/91142/...,https://www.sec.gov/Archives/edgar/data/91142/...


Ho runnato solo i primi 310, poi vanno mergiati tutti insieme

## ---------- VARIABILI IMPORTANTI ED ESEMPI ------------

### VARIABILI IMPORTANTI PER INCOME

Variabile | Descrizione | Frequenza reale di variazione

revenue | Ricavi totali della società | Trimestrale (earnings)

grossProfit | Ricavi - costo del venduto | Trimestrale

operatingIncome | Risultato operativo prima delle tasse e interessi | Trimestrale

netIncome | Utile netto | Trimestrale

eps (Earnings per Share) | Utile per azione | Trimestrale

interestExpense | Interessi passivi | Trimestrale

In [ ]:
# Esempio di output per income

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "revenue": 391035000000,
        "costOfRevenue": 210352000000,
        "grossProfit": 180683000000,
        "grossProfitRatio": 0.4620634982,
        "researchAndDevelopmentExpenses": 31370000000,
        "generalAndAdministrativeExpenses": 0,
        "sellingAndMarketingExpenses": 0,
        "sellingGeneralAndAdministrativeExpenses": 26097000000,
        "otherExpenses": 0,
        "operatingExpenses": 57467000000,
        "costAndExpenses": 267819000000,
        "interestIncome": 0,
        "interestExpense": 0,
        "depreciationAndAmortization": 11445000000,
        "ebitda": 134661000000,
        "ebitdaratio": 0.3443707085,
        "operatingIncome": 123216000000,
        "operatingIncomeRatio": 0.3151022287,
        "totalOtherIncomeExpensesNet": 269000000,
        "incomeBeforeTax": 123485000000,
        "incomeBeforeTaxRatio": 0.3157901467,
        "incomeTaxExpense": 29749000000,
        "netIncome": 93736000000,
        "netIncomeRatio": 0.2397125577,
        "eps": 6.11,
        "epsdiluted": 6.08,
        "weightedAverageShsOut": 15343783000,
        "weightedAverageShsOutDil": 15408095000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER BALANCE

Variabile | Descrizione | Frequenza reale

totalAssets | Totale attivo | Trimestrale

totalLiabilities | Totale passivo | Trimestrale

cashAndCashEquivalents | Cassa disponibile | Trimestrale

totalDebt | Totale debito (a breve e lungo termine) | Trimestrale

inventory | Scorte | Trimestrale

shareholderEquity | Capitale proprio | Trimestrale

In [ ]:
# Esempio di output per balance

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "cashAndCashEquivalents": 29943000000,
        "shortTermInvestments": 35228000000,
        "cashAndShortTermInvestments": 65171000000,
        "netReceivables": 66243000000,
        "inventory": 7286000000,
        "otherCurrentAssets": 14287000000,
        "totalCurrentAssets": 152987000000,
        "propertyPlantEquipmentNet": 45680000000,
        "goodwill": 0,
        "intangibleAssets": 0,
        "goodwillAndIntangibleAssets": 0,
        "longTermInvestments": 91479000000,
        "taxAssets": 19499000000,
        "otherNonCurrentAssets": 55335000000,
        "totalNonCurrentAssets": 211993000000,
        "otherAssets": 0,
        "totalAssets": 364980000000,
        "accountPayables": 68960000000,
        "shortTermDebt": 22511000000,
        "taxPayables": 26601000000,
        "deferredRevenue": 8249000000,
        "otherCurrentLiabilities": 50071000000,
        "totalCurrentLiabilities": 176392000000,
        "longTermDebt": 96548000000,
        "deferredRevenueNonCurrent": 0,
        "deferredTaxLiabilitiesNonCurrent": 0,
        "otherNonCurrentLiabilities": 35090000000,
        "totalNonCurrentLiabilities": 131638000000,
        "otherLiabilities": 0,
        "capitalLeaseObligations": 12430000000,
        "totalLiabilities": 308030000000,
        "preferredStock": 0,
        "commonStock": 83276000000,
        "retainedEarnings": -19154000000,
        "accumulatedOtherComprehensiveIncomeLoss": -7172000000,
        "othertotalStockholdersEquity": 0,
        "totalStockholdersEquity": 56950000000,
        "totalEquity": 56950000000,
        "totalLiabilitiesAndStockholdersEquity": 364980000000,
        "minorityInterest": 0,
        "totalLiabilitiesAndTotalEquity": 364980000000,
        "totalInvestments": 126707000000,
        "totalDebt": 106629000000,
        "netDebt": 76686000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER CASHFLOW

Variabile | Descrizione | Frequenza reale

operatingCashFlow | Flussi di cassa dalle operazioni | Trimestrale

capitalExpenditure | Investimenti in capitale (CAPEX) | Trimestrale

freeCashFlow | Cash flow operativo - CAPEX | Trimestrale

dividendsPaid | Dividendi distribuiti | Trimestrale

In [ ]:
# Esempio di output per cashflow
[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "netIncome": 93736000000,
        "depreciationAndAmortization": 11445000000,
        "deferredIncomeTax": 0,
        "stockBasedCompensation": 11688000000,
        "changeInWorkingCapital": 3651000000,
        "accountsReceivables": -5144000000,
        "inventory": -1046000000,
        "accountsPayables": 6020000000,
        "otherWorkingCapital": 3821000000,
        "otherNonCashItems": -2266000000,
        "netCashProvidedByOperatingActivities": 118254000000,
        "investmentsInPropertyPlantAndEquipment": -9447000000,
        "acquisitionsNet": 0,
        "purchasesOfInvestments": -48656000000,
        "salesMaturitiesOfInvestments": 62346000000,
        "otherInvestingActivites": -1308000000,
        "netCashUsedForInvestingActivites": 2935000000,
        "debtRepayment": -5998000000,
        "commonStockIssued": 0,
        "commonStockRepurchased": -94949000000,
        "dividendsPaid": -15234000000,
        "otherFinancingActivites": -5802000000,
        "netCashUsedProvidedByFinancingActivities": -121983000000,
        "effectOfForexChangesOnCash": 0,
        "netChangeInCash": -794000000,
        "cashAtEndOfPeriod": 29943000000,
        "cashAtBeginningOfPeriod": 30737000000,
        "operatingCashFlow": 118254000000,
        "capitalExpenditure": -9447000000,
        "freeCashFlow": 108807000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]